**Desription**:
    
250bp 

**NOTE:** CrowdVar Size Range

The following notebook is a revised version of the [Crowdvariant analysis notebook](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Machine%20Learning%5BAll%20Technologies%5D.ipynb). 

In the first part of this notebook, a random forest classifier was trained on crowdsourced[CrowdVariant](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Data%20Collection.ipynb) data for HG002 Deletions. 

In the second part of this notebook, the trained randfom forest classifier is then used to predict labels for a new set of randomly selected HG002 Deletions.

In [1]:
"""
Imports
"""
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



***
Load Data
***
Restructure features so that the training dataset and prediction dataset have all of the same features

In [2]:
### Import Data
df_crowd = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_250bp.csv')
### Copy data in new dataframe as a later reference
df_crowd_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_250bp.csv')

In [3]:
df_crowd.head(3)

,chrom,start,end,sample,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label,CN0_prob,CN1_prob,CN2_prob
0,1,187464828,187466479,HG002,0.0,0.000000,0,0.0,0.000000,0,...,8,0.096911,0,0.0,0,0,1,0.00,0.91,0.09
1,1,33156824,33157000,HG002,887.0,44.298984,5,356.4,73.499932,4,...,2,0.221591,0,0.0,0,0,1,0.04,0.91,0.05
2,1,53594099,53595428,HG002,912.2,40.041978,5,381.2,67.957045,5,...,3,0.059443,0,0.0,0,0,0,0.96,0.04,0.00


In [4]:
# Read in HG002 DEL dataframe
HG002_pred = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_250bp.csv')
HG002_pred_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_250bp.csv')

In [5]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(c) - set(d)

{'CN0_prob',
 'CN1_prob',
 'CN2_prob',
 'Ill250.amb_reason_insertSizeScore_insertSizeScore',
 'Label',
 'size'}

In [6]:
# a = ['chrom','start','end','sample','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_insertSizeScore_insertSizeScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','size','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','Label','CN0_prob','CN1_prob','CN2_prob']
# b = ['chrom','start','end','sample','id','type','SVtype','Size','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']

# set(b) - set(a)

In [7]:
### Training Set: Drop irrelevant columns and GT information
df_crowd.drop(['GTcons'], axis=1, inplace = True)
df_crowd.drop(['GTconflict'], axis=1, inplace = True)
df_crowd.drop(['GTsupp'], axis=1, inplace = True)
# df_crowd.drop('SVtype', axis=1)
# df_crowd.drop(['type'],axis=1)
df_crowd.drop(['start'],axis=1, inplace = True)
df_crowd.drop(['end'],axis=1, inplace = True)
df_crowd.drop(['chrom'],axis=1, inplace = True)
# df_crowd.drop('Size',axis=1)
df_crowd.drop(['CN0_prob'],axis=1, inplace = True)
df_crowd.drop(['CN1_prob'],axis=1, inplace = True)
df_crowd.drop(['CN2_prob'],axis=1, inplace = True)
# df_crowd.drop(['TenX.GT'],axis=1, inplace = True)
# df_crowd.drop(['pacbio.GT'],axis=1, inplace = True)
# df_crowd.drop(['IllMP.GT'],axis=1, inplace = True)
df_crowd.drop(['Ill250.GT'],axis=1, inplace = True)
# df_crowd.drop(['Ill300x.GT'],axis=1, inplace = True)
df_crowd.drop(['sample'],axis=1, inplace = True)
# df_crowd.columns.drop(['Label'],axis=1, inplace = True)
df_crowd.drop(['size'],axis=1, inplace = True)
# df_crowd.drop(['Svsize'],axis=1, inplace = True)
df_crowd.drop(['Ill250.amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)

In [8]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(d) - set(c)

{'GIAB_Crowd',
 'GTconflict',
 'GTcons',
 'GTsupp',
 'Ill250.GT',
 'SVtype',
 'Size',
 'chrom',
 'end',
 'id',
 'sample',
 'start',
 'type'}

In [9]:
### Prediction Set: Drop irrelevant columns and GT information
HG002_pred.drop(['GTcons'], axis=1, inplace = True)
HG002_pred.drop(['GTconflict'], axis=1, inplace = True)
HG002_pred.drop(['GTsupp'], axis=1, inplace = True)
HG002_pred.drop(['type'],axis=1, inplace = True)
HG002_pred.drop(['SVtype'],axis=1, inplace = True)
HG002_pred.drop(['start'],axis=1, inplace = True)
HG002_pred.drop(['end'],axis=1, inplace = True)
HG002_pred.drop(['chrom'],axis=1, inplace = True)
HG002_pred.drop(['Size'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.GT'],axis=1, inplace = True)
# HG002_pred.drop(['pacbio.GT'],axis=1, inplace = True)
# HG002_pred.drop(['IllMP.GT'],axis=1, inplace = True)
HG002_pred.drop(['Ill250.GT'],axis=1, inplace = True)
# HG002_pred.drop(['Ill300x.GT'],axis=1, inplace = True)
HG002_pred.drop(['sample'],axis=1, inplace = True)
HG002_pred.drop(['id'],axis=1, inplace = True)
# HG002_pred.drop(['Ill300x.amb_reason_insertSizeScore_orientation'],axis=1, inplace = True)
# HG002_pred.drop(['IllMP.amb_reason_insertSizeScore_alignmentScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP1_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP1_ref_reason_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP2_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP2_amb_reason_orientation_insertSizeScore'],axis=1, inplace = True)

In [10]:
df_crowd.head(3)

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,0.0,0.000000,0,0.0,0.000000,0,0,0,870.053922,144.726287,...,83.074486,145,0,8,0.096911,0,0.0,0,0,1
1,887.0,44.298984,5,356.4,73.499932,4,0,1,841.293839,138.419819,...,122.825237,47,0,2,0.221591,0,0.0,0,0,1
2,912.2,40.041978,5,381.2,67.957045,5,0,0,828.734375,149.026764,...,0.000000,0,0,3,0.059443,0,0.0,0,0,0


*** 
Impute missing values using KNN 
***

In [11]:
# Store data in a new variable which will be converted to a matrix
X = df_crowd
X.head(3)

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,0.0,0.000000,0,0.0,0.000000,0,0,0,870.053922,144.726287,...,83.074486,145,0,8,0.096911,0,0.0,0,0,1
1,887.0,44.298984,5,356.4,73.499932,4,0,1,841.293839,138.419819,...,122.825237,47,0,2,0.221591,0,0.0,0,0,1
2,912.2,40.041978,5,381.2,67.957045,5,0,0,828.734375,149.026764,...,0.000000,0,0,3,0.059443,0,0.0,0,0,0


In [12]:
# NOTE: 250bp dataframe has no missing values


# Convert dataframe to matrix
# X=X.as_matrix()

# #Imput missing values from three closest observations
# X_imputed=KNN(k=3).complete(X)
# X=pd.DataFrame(X_imputed)

In [13]:
# Store header values in a list, will be used later to re-label the matrix post KNN imputation
# dfcrowd_header = list(df_crowd.columns.values)

In [14]:
# X.columns = dfcrowd_header
# X.head(3)

In [15]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y['Label'] = X['Label']
#Y = X.pop('Label')

In [16]:
#Count the number of labels
pd.value_counts(Y['Label'].values, sort=False)

 0    552
 1    959
 2      3
-1      1
dtype: int64

In [17]:
# Remove labels from feature set
X.drop(['Label'],axis=1, inplace = True)

In [18]:
# Order features
X4 = X.reindex_axis(sorted(X.columns), axis=1)

***
Machine Learning
***

** Train RF Classifier **

In [19]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X4, Y, test_size=0.3, random_state=0)

In [20]:
# Train Random Forest Classifier
model = RandomForestClassifier() 
model.fit(X_train, y_train)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [21]:
#NOTE: Training Set - Show number of Hom Ref, Hom Var, Het Var datapoints the model was trained on
ytrain = pd.DataFrame()
ytrain['ytrain'] = y_train
pd.value_counts(ytrain['ytrain'].values, sort=False)

 1.0    671
 0.0     73
-1.0      1
 2.0      2
dtype: int64

** Precision Score **
- Overall model performance
- Using 30% of original dataset (test set)
- Truth labels: CrowdVariant labels

In [22]:
model.predict(X_test)

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1,

In [23]:
pred = model.predict(X_test)

In [24]:
precision_score(pred, y_test, average='micro') 

0.9758241758241758

In [25]:
# Add original labels and predicted labels back to the original dataframe
df_Xtest = pd.DataFrame(X_test)
df_Xtest.head()

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,Ill250.ref_insertSize_mean,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,refN_cnt,refN_pct,segdup_cnt,segdup_pct,tandemrep_cnt,tandemrep_pct
1222,994.916667,5.415384,48,447.020833,92.768864,44,4,0,904.924419,131.254148,...,0.000000,0.000000,0,0,0,0,0,0.000000,1,0.12500
310,970.742857,26.804416,35,460.485714,79.329465,34,1,0,881.921162,147.513942,...,452.300000,109.609808,20,0,0,0,0,0.000000,1,0.10625
9,0.000000,0.000000,0,0.000000,0.000000,0,0,0,886.094675,150.269472,...,441.163636,73.698591,55,0,0,0,0,0.000000,0,0.00000
785,978.500000,21.362350,20,425.550000,75.699719,19,1,0,903.137725,153.256269,...,434.233333,99.281648,30,0,0,0,1,0.404959,1,0.31405
295,955.318182,20.636213,22,446.727273,93.222696,21,1,0,894.035503,145.256675,...,407.884615,77.514429,26,0,0,0,0,0.000000,2,0.16358


In [26]:
labels = pd.DataFrame(y_test)

In [27]:
df_crowd.head(3)

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,Ill250.ref_insertSize_mean,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct
0,0.0,0.000000,0,0.0,0.000000,0,0,0,870.053922,144.726287,...,429.172414,83.074486,145,0,8,0.096911,0,0.0,0,0
1,887.0,44.298984,5,356.4,73.499932,4,0,1,841.293839,138.419819,...,488.702128,122.825237,47,0,2,0.221591,0,0.0,0,0
2,912.2,40.041978,5,381.2,67.957045,5,0,0,828.734375,149.026764,...,0.000000,0.000000,0,0,3,0.059443,0,0.0,0,0


In [28]:
df_Xtest['predicted_label'] = pred
df_Xtest['Label'] = df_crowd_2['Label']
df_Xtest['chrom'] = df_crowd_2['chrom']
df_Xtest['start'] = df_crowd_2['start']
df_Xtest['end'] = df_crowd_2['end']
# df_Xtest['Y_test'] = labels

In [29]:
pd.value_counts(df_Xtest['Label'].values, sort=False)

0    166
1    288
2      1
dtype: int64

In [30]:
pd.value_counts(df_Xtest['predicted_label'].values, sort=False)

0    166
1    289
dtype: int64

In [31]:
from sklearn.metrics import confusion_matrix
ytest = df_Xtest['Label']
predict = df_Xtest['predicted_label']
print(confusion_matrix(ytest, predict))

[[161   5   0]
 [  5 283   0]
 [  0   1   0]]


In [32]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,161,5,166
1,5,283,288
2,0,1,1
All,166,289,455


In [33]:
from sklearn.metrics import classification_report
print(classification_report(ytest, predict))

             precision    recall  f1-score   support

          0       0.97      0.97      0.97       166
          1       0.98      0.98      0.98       288
          2       0.00      0.00      0.00         1

avg / total       0.97      0.98      0.97       455



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



***
Predict
***

**Load Data**

In [34]:
HG002_pred.head(3)

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,0.0,0.0,0,0.0,0.0,0,0,0,835.040323,155.109139,...,170.701012,23,0,0,0.0,1,0.679775,0,0,2
1,0.0,0.0,0,0.0,0.0,0,0,0,887.524430,128.971054,...,101.977510,30,4,0,0.0,1,1.000000,0,0,2
2,0.0,0.0,0,0.0,0.0,0,0,0,884.496970,167.733072,...,80.711669,55,0,0,0.0,0,0.000000,0,0,2


In [35]:
X2 = HG002_pred

** Impute missing values using KNN **

In [36]:
# # NOTE: 250bp dataframe has no missing values

# # Convert dataframe to matrix
# X2=X2.as_matrix()

# #Imput missing values from three closest observations
# X2_imputed=KNN(k=3).complete(X2)
# X2=pd.DataFrame(X2_imputed)

In [37]:
# X2.columns = list(HG002_pred.columns.values)

In [38]:
X2.drop(['GIAB_Crowd'],axis=1, inplace = True)

In [39]:
# Order features
X5 = X2.reindex_axis(sorted(X2.columns), axis=1)

In [40]:
model.predict(X5)

array([1, 1, 1, ..., 0, 0, 0])

In [41]:
# Calculate Precision Score
true = HG002_pred_2['GIAB_Crowd']
predicted = model.predict(X5)
precision_score(true, predicted, average='micro') 

0.78395721925133688

In [42]:
pd.crosstab(true, predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,-1,0,1,All
True,,,,
0,0,594,4,598
1,1,128,1605,1734
2,1,21,451,473
All,2,743,2060,2805


In [43]:
from sklearn.metrics import classification_report
true = HG002_pred_2['GIAB_Crowd']
pred = model.predict(X5)
print(classification_report(true, pred))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          0       0.80      0.99      0.89       598
          1       0.78      0.93      0.85      1734
          2       0.00      0.00      0.00       473

avg / total       0.65      0.78      0.71      2805



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [44]:
pred_prob = model.predict_proba(X5)

In [45]:
X5['predicted_label'] = pred
X5['Label'] = HG002_pred_2['GIAB_Crowd']
X5['chrom'] = HG002_pred_2['chrom']
X5['start'] = HG002_pred_2['start']
X5['end'] = HG002_pred_2['end']
X5['Size'] = HG002_pred_2['Size']
X5['GTcons'] = HG002_pred_2['GTcons']

In [46]:
X6 = pd.concat([X5, pd.DataFrame(pred_prob, columns=['1','2','3','4'])])

In [47]:
X6.to_csv('250bp_pred_prob.csv', index=False)

In [48]:
X6 = pd.read_csv('250bp_pred_prob.csv')

In [49]:
X6.rename(columns={'1': 'unknown'}, inplace=True)
X6.rename(columns={'2': 'Hom_Var'}, inplace=True)
X6.rename(columns={'3': 'Het_Var'}, inplace=True)
X6.rename(columns={'4': 'Hom_Ref'}, inplace=True)

In [50]:
X6.rename(columns={'predicted_label': '250bp_predicted_label'}, inplace=True)

In [52]:
X6.to_csv('250bp_pred_prob.csv', index=False)

In [51]:
X6.head(3)

,unknown,Hom_Var,Het_Var,Hom_Ref,GTcons,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,...,chrom,end,250bp_predicted_label,refN_cnt,refN_pct,segdup_cnt,segdup_pct,start,tandemrep_cnt,tandemrep_pct
0,0.0,0.2,0.8,0.0,0,0.0,0.0,0,0.0,0.0,...,1,4137420,1,0,0,1,0.679775,4137242,0,0.0
1,0.0,0.2,0.8,0.0,0,0.0,0.0,0,0.0,0.0,...,1,52270310,1,0,0,1,1.000000,52270083,0,0.0
2,0.0,0.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,...,1,218845886,1,0,0,0,0.000000,218845853,0,0.0


** Show Size Distribution of Labels with Predict Probability > 0.8 **

In [ ]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)

In [ ]:
X_homVar_conf = X6[(X6['Hom_Var'] == 0.9) | (X6['Hom_Var'] == 1) | (X6['Hom_Var'] == 0.8)]

In [ ]:
X_homVar_conf['Size'] = HG002_pred_2['Size'].abs()
X_homVar_conf['Size_Log'] = np.log10(X_homVar_conf.Size)

In [ ]:
output_notebook()

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_conf, values='Size_Log', title='Homozygous Variant: Predict Proba > 0.8', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("250bp_HomVar_PredProb0.8+.html")
show(p)

![Figure 1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/250bp_Hom.png)

In [ ]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_conf = X6[(X6['Het_Var'] == 0.9) | (X6['Het_Var'] == 1) | (X6['Het_Var'] == 0.8)]
X_hetVar_conf['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_conf['Size_Log'] = np.log10(X_hetVar_conf.Size)

In [ ]:
output_notebook()

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_conf, values='Size_Log', title='Heterozygous Variant: Predict Proba > 0.9', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("250bp_HetVar_PredProb0.9+.html")
show(p)

![Figure 1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/250bp_HetVar_PredProb0.9%2B.png)

** Show Size Distribution of Labels with Predict Probability < 0.3 **

In [ ]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)
X_homVar_least = X6[(X6['Hom_Var'] == 0.3) | (X6['Hom_Var'] == 0.2) | (X6['Hom_Var'] == 0.1)]
X_homVar_least['Size'] = HG002_pred_2['Size'].abs()
X_homVar_least['Size_Log'] = np.log10(X_homVar_least.Size)

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_least, values='Size_Log', title='Homozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("250bp_HomVar_PredProb0.3-.html")
show(p)

![Figure 1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/250bp_HomVar_PredProb0.3-.png)

In [ ]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_least = X6[(X6['Het_Var'] == 0.3) | (X6['Het_Var'] == 0.2) | (X6['Het_Var'] == 0.1)]
X_hetVar_least['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_least['Size_Log'] = np.log10(X_hetVar_least.Size)

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_least, values='Size_Log', title='Heterozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("250bp_HetVar_PredProb0.3-.html")
show(p)

![Figure 1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/250bp_HetVar_PredProb0.3-.png)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/crowdVar_size.png)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/svviz_size.png)

In [ ]:
# df_crowd_2['size'] = df_crowd_2['size'].abs()
df_crowd_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_250bp.csv')
df_crowd_2['Size_Log'] = np.log10(df_crowd_2.size)

p = figure()
p = Histogram(df_crowd_2, values='size', title='CrowdVariant Data Size Distribution', color='LightSlateGray', bins=30, xlabel="Size(bp)", ylabel="Frequency")
output_file("CrowdVar.Size.Log.html")
show(p)

**TODO**

[Feature Importance](http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)